In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
benign_dos_csv = "/content/drive/My Drive/EL_SEMESTER_8/VAE/4type/csv_dec/train_benign_for_dos.csv"
benign_port_csv = "/content/drive/My Drive/EL_SEMESTER_8/VAE/4type/csv_dec/train_benign_for_port.csv"
benign_ddos_csv = "/content/drive/My Drive/EL_SEMESTER_8/VAE/4type/csv_dec/train_benign_for_ddos.csv"
benign_patator_csv = "/content/drive/My Drive/EL_SEMESTER_8/VAE/4type/csv_dec/train_benign_for_patator.csv"

In [ ]:
detected_dos_csv = "/content/drive/My Drive/EL_SEMESTER_8/VAE/4type/csv_dec/train_dos.csv"
detected_port_csv = "/content/drive/My Drive/EL_SEMESTER_8/VAE/4type/csv_dec/train_portscan.csv"
detected_ddos_csv = "/content/drive/My Drive/EL_SEMESTER_8/VAE/4type/csv_dec/train_ddos.csv"
detected_patator_csv = "/content/drive/My Drive/EL_SEMESTER_8/VAE/4type/csv_dec/train_patator.csv"

In [ ]:
benign_dos_save = "/content/drive/My Drive/EL_SEMESTER_8/VAE/4type/txt_hex/train_benign_for_dos_hex.txt"
benign_port_save = "/content/drive/My Drive/EL_SEMESTER_8/VAE/4type/txt_hex/train_benign_for_port_hex.txt"
benign_ddos_save = "/content/drive/My Drive/EL_SEMESTER_8/VAE/4type/txt_hex/train_benign_for_ddos_hex.txt"
benign_patator_save = "/content/drive/My Drive/EL_SEMESTER_8/VAE/4type/txt_hex/train_benign_for_patator_hex.txt"

In [ ]:
detected_dos_save = "/content/drive/My Drive/EL_SEMESTER_8/VAE/4type/txt_hex/train_dos_hex.txt"
detected_port_save = "/content/drive/My Drive/EL_SEMESTER_8/VAE/4type/txt_hex/train_portscan_hex.txt"
detected_ddos_save = "/content/drive/My Drive/EL_SEMESTER_8/VAE/4type/txt_hex/train_ddos_hex.txt"
detected_patator_save = "/content/drive/My Drive/EL_SEMESTER_8/VAE/4type/txt_hex/train_patator_hex.txt"

In [ ]:
mix_benign_dos = "/content/drive/My Drive/EL_SEMESTER_8/VAE/4type/txt_hex/mix_benign_dos.txt"
mix_benign_port = "/content/drive/My Drive/EL_SEMESTER_8/VAE/4type/txt_hex/mix_benign_port.txt"
mix_benign_ddos = "/content/drive/My Drive/EL_SEMESTER_8/VAE/4type/txt_hex/mix_benign_ddos.txt"
mix_benign_patator = "/content/drive/My Drive/EL_SEMESTER_8/VAE/4type/txt_hex/mix_benign_patator.txt"

In [ ]:
dos_wb = "/content/drive/My Drive/EL_SEMESTER_8/VAE/4type/txt_wb/input_buffer_dos.txt"
port_wb = "/content/drive/My Drive/EL_SEMESTER_8/VAE/4type/txt_wb/input_buffer_port.txt"
ddos_wb = "/content/drive/My Drive/EL_SEMESTER_8/VAE/4type/txt_wb/input_buffer_ddos.txt"
patator_wv = "/content/drive/My Drive/EL_SEMESTER_8/VAE/4type/txt_wb/input_buffer_patator.txt"

In [ ]:
import csv

In [ ]:
def decimal_to_fixed_point(value, integer_bits=5, fractional_bits=10):
    """
    Mengonversi nilai desimal ke representasi fixed-point dalam format Q5.11.

    Args:
        value (float): Nilai desimal yang akan dikonversi.
        integer_bits (int): Jumlah bit untuk bagian integer.
        fractional_bits (int): Jumlah bit untuk bagian fraksional.

    Returns:
        str: Representasi fixed-point dalam bentuk biner.
        int: Nilai fixed-point dalam bentuk bilangan bulat.
        str: Representasi fixed-point dalam bentuk heksadesimal.
    """
    # Total bit: 1 sign bit + integer_bits + fractional_bits
    total_bits = 1 + integer_bits + fractional_bits  # Total 16 bit untuk Q5.11

    # Faktor skala (untuk bagian fraksional)
    scale_factor = 2 ** fractional_bits

    # Konversi nilai desimal ke fixed-point integer
    fixed_point_value = int(round(value * scale_factor))

    # Rentang nilai untuk format Q5.11
    min_value = -(2 ** (total_bits - 1))  # Rentang nilai minimum
    max_value = (2 ** (total_bits - 1)) - 1  # Rentang nilai maksimum
    if fixed_point_value < min_value or fixed_point_value > max_value:
        raise ValueError(f"Value {value} is out of range for Q{integer_bits+1}.{fractional_bits} format.")

    # Jika negatif, lakukan two's complement secara manual
    if fixed_point_value < 0:
        fixed_point_value = (1 << total_bits) + fixed_point_value  # Two's complement

    # Representasi biner 16-bit
    binary_representation = f"{fixed_point_value:016b}"

    # Representasi heksadesimal 4-digit (16-bit)
    hex_representation = f"{fixed_point_value:04X}"

    return binary_representation, fixed_point_value, hex_representation

In [ ]:
def process_csv_to_hex64(input_csv, output_txt):
    with open(input_csv, newline='') as csvfile:
        reader = csv.reader(csvfile)
        rows = list(reader)  # Simpan semua baris

    processed_rows = []
    for row in rows[1:]:  # Melewati header
        hex_values = [decimal_to_fixed_point(float(value))[2] for value in row]  # Ambil nilai hex saja
        hex_64bit_values = [f"{hex_val}000000000000" for hex_val in hex_values]  # Pad jadi 64-bit
        processed_rows.append(", ".join(hex_64bit_values))

    with open(output_txt, "w") as outfile:
        outfile.write("\n".join(processed_rows))

def merge_txt(file1, file2, a, b, output_file):
    with open(file1, 'r') as f1, open(file2, 'r') as f2, open(output_file, 'w') as out:
        lines1 = [next(f1, '').strip() for _ in range(a)]
        lines2 = [next(f2, '').strip() for _ in range(b)]

        # Write lines to output file
        out.write('\n'.join(lines1 + lines2) + '\n')


In [ ]:
# Proses file CSV
process_csv_to_hex64(benign_dos_csv, benign_dos_save)
print(f"Konversi selesai. Hasil disimpan di {benign_dos_save}")

process_csv_to_hex64(benign_port_csv, benign_port_save)
print(f"Konversi selesai. Hasil disimpan di {benign_port_save}")

process_csv_to_hex64(benign_ddos_csv, benign_ddos_save)
print(f"Konversi selesai. Hasil disimpan di {benign_ddos_save}")

process_csv_to_hex64(benign_patator_csv, benign_patator_save)
print(f"Konversi selesai. Hasil disimpan di {benign_patator_save}")

Konversi selesai. Hasil disimpan di /content/drive/My Drive/EL_SEMESTER_8/VAE/4type/txt_hex/train_benign_for_dos_hex.txt
Konversi selesai. Hasil disimpan di /content/drive/My Drive/EL_SEMESTER_8/VAE/4type/txt_hex/train_benign_for_port_hex.txt
Konversi selesai. Hasil disimpan di /content/drive/My Drive/EL_SEMESTER_8/VAE/4type/txt_hex/train_benign_for_ddos_hex.txt
Konversi selesai. Hasil disimpan di /content/drive/My Drive/EL_SEMESTER_8/VAE/4type/txt_hex/train_benign_for_patator_hex.txt


In [ ]:
process_csv_to_hex64(detected_dos_csv, detected_dos_save)
print(f"Konversi selesai. Hasil disimpan di {detected_dos_save}")

process_csv_to_hex64(detected_port_csv, detected_port_save)
print(f"Konversi selesai. Hasil disimpan di {detected_port_save}")

process_csv_to_hex64(detected_ddos_csv, detected_ddos_save)
print(f"Konversi selesai. Hasil disimpan di {detected_ddos_save}")

process_csv_to_hex64(detected_patator_csv, detected_patator_save)
print(f"Konversi selesai. Hasil disimpan di {detected_patator_save}")

Konversi selesai. Hasil disimpan di /content/drive/My Drive/EL_SEMESTER_8/VAE/4type/txt_hex/train_dos_hex.txt
Konversi selesai. Hasil disimpan di /content/drive/My Drive/EL_SEMESTER_8/VAE/4type/txt_hex/train_portscan_hex.txt
Konversi selesai. Hasil disimpan di /content/drive/My Drive/EL_SEMESTER_8/VAE/4type/txt_hex/train_ddos_hex.txt
Konversi selesai. Hasil disimpan di /content/drive/My Drive/EL_SEMESTER_8/VAE/4type/txt_hex/train_patator_hex.txt


In [ ]:
# Contoh pemanggilan fungsi
merge_txt(benign_dos_save, detected_dos_save, 1000, 1000, mix_benign_dos)
merge_txt(benign_port_save, detected_port_save, 1000, 1000, mix_benign_port)
merge_txt(benign_ddos_save, detected_ddos_save, 2000, 2000, mix_benign_ddos)
merge_txt(benign_patator_save, detected_patator_save, 1000, 1000, mix_benign_patator)

In [ ]:
def load_file1(file1_path):
    """Membaca file pertama dan menyimpannya sebagai list berisi 27 elemen."""
    with open(file1_path, 'r') as f:
        file1_data = [line.strip() for line in f.readlines()]
    return file1_data


def load_file2(file2_path, max_lines=50):
    """Membaca file kedua dan mengembalikan daftar dari daftar (list of lists), dibatasi max_lines baris."""
    with open(file2_path, 'r') as f:
        file2_data = [line.strip().split(', ') for line in f.readlines()[:max_lines]]
    return file2_data


def combine_files(file1_data, file2_data):
    """Menggabungkan data dari file pertama dengan setiap baris dari file kedua."""
    buffer_n = []
    for row in file2_data:
        buffer_n.append(file1_data + row)  # Gabungkan 27 elemen dari file1 dan 9 dari file2
    return buffer_n

def print_formatted_output(buffer_n):
    """Mencetak output dengan format yang lebih mudah dibaca."""
    for i, row in enumerate(buffer_n):
        print(f"Array {i + 1}:")
        for j in range(0, len(row), 9):  # Cetak dalam kelompok 9 elemen
            print(" ".join(row[j:j+9]))
        print()

# Proses
file1_data = load_file1(dos_wb)
file2_data = load_file2(mix_benign_dos, max_lines=50)
buffer_n = combine_files(file1_data, file2_data)

print("Data berhasil digabungkan dalam bentuk array:")
print_formatted_output(buffer_n)


Data berhasil digabungkan dalam bentuk array:
Array 1:
FEDC013EFD82FE09 020CFF7900000000 1BA215C800000000 F078F3C100000000 F869FA0400000000 06B8054D00000000 0194013F00000000 11DB0DE400000000 E99CEE8600000000
FD31FC9BDA04D9FE FEA8FF9000000000 FF23FD7B00000000 0149012B00000000 FDC2FE6700000000 FC6DFB1D00000000 FEDC011000000000 FDCAFE7C00000000 FBFAFC1600000000
071505F201400000 075505A101400000 075D059701400000 074605B401400000 074F05A801400000 072705DB01400000 072805DA01400000 076B058501400000 073F05BD01400000
0086000000000000 0086000000000000 0060000000000000 004C000000000000 0073000000000000 0055000000000000 0028000000000000 0055000000000000 0055000000000000

Array 2:
FEDC013EFD82FE09 020CFF7900000000 1BA215C800000000 F078F3C100000000 F869FA0400000000 06B8054D00000000 0194013F00000000 11DB0DE400000000 E99CEE8600000000
FD31FC9BDA04D9FE FEA8FF9000000000 FF23FD7B00000000 0149012B00000000 FDC2FE6700000000 FC6DFB1D00000000 FEDC011000000000 FDCAFE7C00000000 FBFAFC1600000000
071505F201400000 

In [ ]:
def fixed_point_to_decimal(value, integer_bits=5, fractional_bits=10):
    """
    Mengonversi bilangan biner/heksadesimal dalam format Q5.10 ke desimal.

    Args:
        value (str): Nilai dalam format biner (16-bit) atau heksadesimal (4-digit).
        integer_bits (int): Jumlah bit untuk bagian integer.
        fractional_bits (int): Jumlah bit untuk bagian fraksional.

    Returns:
        float: Nilai desimal yang dikonversi dari fixed-point.
    """
    # Total bit dalam representasi fixed-point
    total_bits = 1 + integer_bits + fractional_bits  # Total 16 bit untuk Q5.10

    # Jika input dalam format heksadesimal, konversi ke bilangan bulat
    if isinstance(value, str) and len(value) == 4:  # Heksadesimal (misal: '000C')
        fixed_point_value = int(value, 16)
    elif isinstance(value, str) and len(value) == 16:  # Biner (misal: '0000000000001100')
        fixed_point_value = int(value, 2)
    else:
        raise ValueError("Input harus berupa string biner (16-bit) atau heksadesimal (4-digit).")

    # Jika bilangan dalam two's complement (negatif)
    if fixed_point_value >= (1 << (total_bits - 1)):  # Cek bit tanda (bit MSB)
        fixed_point_value -= (1 << total_bits)  # Konversi dari two's complement

    # Konversi ke desimal
    scale_factor = 2 ** fractional_bits
    decimal_value = fixed_point_value / scale_factor

    return decimal_value


# Contoh penggunaan:
a_hex = ['000C', '0000', '004D', '0115']  # Contoh input dalam format heksadesimal
a_decimal = [fixed_point_to_decimal(val) for val in a_hex]

print(a_decimal)  # Output: [0.0117, 0.0, 0.0752, 0.2705]

[0.01171875, 0.0, 0.0751953125, 0.2705078125]
